In [2]:
import pandas as pd
import numpy as np
from ast import literal_eval
from pypif import pif
from pymatgen import Composition
from csv_template_ingester.converter import convert

def convert_df_to_pif(df_csv_file_path):
    
    pifs = [x for x in convert(files=[df_csv_file_path])]
    pif_outfile_path = df_csv_file_path.replace('.csv', '.json')
    with open(pif_outfile_path, 'w') as output_file:
        pif.dump(pifs, output_file)

        
    return pifs, pif_outfile_path

In [3]:
df = pd.read_csv('training_data_modified.csv')

In [4]:
# slice unnecessary feature columns for Citrination
df['PROPERTY: element A'] = df['formulaA']
df['PROPERTY: element B'] = df['formulaB']
df['PROPERTY: stabilityVec'] = df['stabilityVec']
df_citrination = df.filter(['FORMULA', 'PROPERTY: element A', 'PROPERTY: element B', 'PROPERTY: binary_classifier', 'PROPERTY: stabilityVec'], axis=1)
outfile_path = 'training_data_no_features.csv'
# df_citrination = df_citrination.sample(5000)
print(len(df_citrination))
df_citrination.to_csv(outfile_path)


28292


In [6]:
# convert to pif for quicker upload
# convert_df_to_pif(outfile_path)

In [26]:
from os import environ
from pypif import pif
import itertools
from citrination_client import *

# establish client (api key stored as enviornment variable)
client = CitrinationClient(environ['CITRINATION_API_KEY'], 'https://citrination.com')

# example predition
resp = client.models.predict('11942', [{'formula':'NaCl'},])
prediction = resp[0].get_value('Property binary_classifier')
print(prediction.value, prediction.loss)

unstable 0.5238095238095237


In [41]:
df_test = pd.read_csv('test_data_modified.csv')

candidates = [{'formula':i} for i in df_test['FORMULA'].values]
print(candidates[0:2])
print(len(candidates))

[{'formula': 'Ac1'}, {'formula': 'Ac0.9 Be0.1'}]
8239


In [42]:
batches = [candidates[x:x+1000] for x in range(0, len(candidates), 1000)]
print(len(batches), len(batches[-1]))

9 239


In [43]:
import csv
dv_id = '11942'
view = client.models.get_data_view(dv_id)

predictions = []
prediction_errors = []
for batch in batches:
    print(len(batch))
    print(batch[0])
    resp = client.models.predict(dv_id, batch)
    predictions.extend([r.get_value('Property binary_classifier').value for r in resp])
    prediction_errors.extend([r.get_value('Property binary_classifier').loss for r in resp])

print(predictions[0])

1000
{'formula': 'Ac1'}
1000
{'formula': 'Ag1'}
1000
{'formula': 'Eu0.1 Ge0.9'}
1000
{'formula': 'Lu0.8 In0.2'}
1000
{'formula': 'Na0.3 Ga0.7'}
1000
{'formula': 'Pm0.4 Mg0.6'}
1000
{'formula': 'Na0.5 Sb0.5'}
1000
{'formula': 'Cs0.4 Te0.6'}
239
{'formula': 'Yb0.7 Se0.3'}
stable


In [44]:
print(len(predictions))
print(predictions[0])

8239
stable


In [45]:
df_test['Predicted Stabilityvec'] = predictions
df_test['Predicted Stabilityvec error'] = prediction_errors

In [46]:
df_test.to_csv('test_data_modified_with_predicitions.csv')

In [52]:
stabilityVec = []
temp_vec = []
count = 0
for p in predictions:
    if p == 'unstable':
        i = 0
    else:
        i = 1
    temp_vec.append(i)
    count += 1
    if count == 11:
        stabilityVec.append(temp_vec)
        temp_vec = []
        count = 0
print(stabilityVec[0:3])
print(len(stabilityVec))

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
749
